<a href="https://colab.research.google.com/github/diyism/colab_kaldi2/blob/main/my_icefall_yes_no_dataset_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1. Install PyTorch, torchaudio, k2
import torch
print(torch.__version__)

!pip install -q torchaudio
import torchaudio
print(torchaudio.__version__)

#the cuda version and torch version should match the upper printed verions
!pip install -q k2==1.24.4.dev20240905+cuda12.1.torch2.4.1 -f https://k2-fsa.github.io/k2/cuda.html
!python3 -m k2.version

2.4.1+cu121
2.4.1+cu121

k2 version: 1.24.4
Build type: Release
Git SHA1: cf664841c6d93e21e59b40aade84869b76c919c1
Git date: Thu Sep 5 19:25:17 2024
Cuda used to build k2: 12.1
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.30.2
GCC version: 9.3.1
CMAKE_CUDA_FLAGS: -Wno-deprecated-gpu-targets -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_50,code=sm_50 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_60,code=sm_60 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_61,code=sm_61 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_70,code=sm_70 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_75,code=sm_75 -lineinfo --expt-extended

In [ ]:
#2. install lhotse
#Normally, we would use pip install lhotse. However, the yesno recipe is added recently and has not been released to PyPI yet,
#so we install the latest unreleased version here.
!pip install -q git+https://github.com/lhotse-speech/lhotse

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#3. install icefall
#k2-icefall is a collection of Python scripts.You don't need to install it. What you need to do is to get its source code,
#install its dependencies, and set the PYTHONPATH pointing to it.
!git clone https://github.com/k2-fsa/icefall
!pip install -q --upgrade onnxconverter-common
!cd icefall && pip install -q -r requirements.txt

Cloning into 'icefall'...
remote: Enumerating objects: 18395, done.
remote: Counting objects: 100% (703/703), done.
remote: Compressing objects: 100% (384/384), done.
remote: Total 18395 (delta 373), reused 548 (delta 278), pack-reused 17692 (from 1)
Receiving objects: 100% (18395/18395), 20.29 MiB | 8.88 MiB/s, done.
Resolving deltas: 100% (12469/12469), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 92.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python

In [ ]:
#4. data preparation

# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  rm -rf data && \
  ./prepare.sh

2024-09-23 16:19:28 (prepare.sh:27:main) dl_dir: /content/icefall/egs/yesno/ASR/download
2024-09-23 16:19:28 (prepare.sh:30:main) Stage 0: Download data
/content/icefall/egs/yesno/ASR/download/waves_yesno.tar.gz: 100% 4.70M/4.70M [00:01<00:00, 3.52MB/s]
2024-09-23 16:19:34 (prepare.sh:39:main) Stage 1: Prepare yesno manifest
2024-09-23 16:19:37 (prepare.sh:45:main) Stage 2: Compute fbank for yesno
2024-09-23 16:19:41,254 INFO [compute_fbank_yesno.py:65] Processing train
Extracting and storing features: 100% 90/90 [00:00<00:00, 145.99it/s]
2024-09-23 16:19:41,893 INFO [compute_fbank_yesno.py:65] Processing test
Extracting and storing features: 100% 30/30 [00:00<00:00, 275.78it/s]
2024-09-23 16:19:42 (prepare.sh:51:main) Stage 3: Prepare lang
2024-09-23 16:19:48,195 INFO [prepare_lang_fst.py:174] Building standard CTC topology
2024-09-23 16:19:48,196 INFO [prepare_lang_fst.py:183] Building L
2024-09-23 16:19:48,197 INFO [prepare_lang_fst.py:191] Building HL
2024-09-23 16:19:48,198 INFO [

In [ ]:
#5.training

! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/train.py

2024-09-23 16:29:58,170 INFO [train.py:481] Training started
2024-09-23 16:29:58,170 INFO [train.py:482] {'exp_dir': PosixPath('tdnn/exp'), 'lang_dir': PosixPath('data/lang_phone'), 'lr': 0.01, 'feature_dim': 23, 'weight_decay': 1e-06, 'start_epoch': 0, 'best_train_loss': inf, 'best_valid_loss': inf, 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 10, 'reset_interval': 20, 'valid_interval': 10, 'beam_size': 10, 'reduction': 'sum', 'use_double_scores': True, 'world_size': 1, 'master_port': 12354, 'tensorboard': True, 'num_epochs': 15, 'seed': 42, 'feature_dir': PosixPath('data/fbank'), 'max_duration': 30.0, 'bucketing_sampler': False, 'num_buckets': 10, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': False, 'return_cuts': True, 'num_workers': 2, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': 'cf664841c6d93e21e59b40aade84869b76c919c1', 'k2-git-

In [ ]:
#6. decode a single sound file
!pip install kaldifeat==1.25.5.dev20240914+cuda12.1.torch2.4.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/pretrained.py \
    --checkpoint /content/icefall/egs/yesno/ASR/tdnn/exp/epoch-14.pt \
    --words-file /content/icefall/egs/yesno/ASR/data/lang_phone/words.txt \
    --HLG /content/icefall/egs/yesno/ASR/data/lang_phone/HLG.pt \
    /content/icefall/egs/yesno/ASR/download/waves_yesno/0_0_1_0_1_0_0_1.wav

Looking in links: https://csukuangfj.github.io/kaldifeat/cuda.html
2024-09-23 16:50:45,387 INFO [pretrained.py:136] {'feature_dim': 23, 'num_classes': 4, 'sample_rate': 8000, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': '/content/icefall/egs/yesno/ASR/tdnn/exp/epoch-14.pt', 'words_file': '/content/icefall/egs/yesno/ASR/data/lang_phone/words.txt', 'HLG': '/content/icefall/egs/yesno/ASR/data/lang_phone/HLG.pt', 'sound_files': ['/content/icefall/egs/yesno/ASR/download/waves_yesno/0_0_1_0_1_0_0_1.wav']}
2024-09-23 16:50:45,407 INFO [pretrained.py:142] device: cuda:0
2024-09-23 16:50:45,407 INFO [pretrained.py:144] Creating model
/content/icefall/egs/yesno/ASR/./tdnn/pretrained.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary

In [ ]:
#7. decode all waves
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/decode.py

! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/recogs-test_set.txt

2024-09-23 16:51:54,314 INFO [decode.py:262] Decoding started
2024-09-23 16:51:54,314 INFO [decode.py:263] {'exp_dir': PosixPath('tdnn/exp'), 'lang_dir': PosixPath('data/lang_phone'), 'feature_dim': 23, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'epoch': 14, 'avg': 2, 'export': False, 'feature_dir': PosixPath('data/fbank'), 'max_duration': 30.0, 'bucketing_sampler': False, 'num_buckets': 10, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': False, 'return_cuts': True, 'num_workers': 2, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': 'cf664841c6d93e21e59b40aade84869b76c919c1', 'k2-git-date': 'Thu Sep 5 19:25:17 2024', 'lhotse-version': '1.28.0.dev+git.bc2c0a2.clean', 'torch-version': '2.4.1+cu121', 'torch-cuda-available': True, 'torch-cuda-version': '12.1', 'python-version': '3.10', 'icefall-git-branch': 'maste

In [ ]:
#8. show the detailed WER
! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/errs-test_set.txt

%WER = 0.42
Errors: 0 insertions, 1 deletions, 0 substitutions, over 240 reference words (239 correct)
Search below for sections starting with PER-UTT DETAILS:, SUBSTITUTIONS:, DELETIONS:, INSERTIONS:, PER-WORD STATS:

PER-UTT DETAILS: corr or (ref->hyp)  
0_0_0_1_0_0_0_1-0:	NO NO NO YES NO NO NO YES
0_0_1_0_0_0_1_0-1:	NO NO YES NO NO NO YES NO
0_0_1_0_0_1_1_1-2:	NO NO YES NO NO YES YES YES
0_0_1_0_1_0_0_1-3:	NO NO YES NO YES NO NO YES
0_0_1_1_0_0_0_1-4:	NO NO YES YES NO NO NO YES
0_0_1_1_0_1_1_0-5:	NO NO YES YES NO YES YES NO
0_0_1_1_1_0_0_0-6:	NO NO YES YES YES NO NO NO
0_0_1_1_1_1_0_0-7:	NO NO YES YES YES YES NO NO
0_1_0_0_0_1_0_0-8:	NO YES NO NO NO YES NO NO
0_1_0_0_1_0_1_0-9:	NO YES NO NO YES NO YES NO
0_1_0_1_0_0_0_0-10:	NO YES NO YES NO NO NO (NO->*)
0_1_0_1_1_1_0_0-11:	NO YES NO YES YES YES NO NO
0_1_1_0_0_1_1_1-12:	NO YES YES NO NO YES YES YES
0_1_1_1_0_0_1_0-13:	NO YES YES YES NO NO YES NO
0_1_1_1_1_0_1_0-14:	NO YES YES YES YES NO YES NO
1_0_0_0_0_0_0_0-15:	YES NO NO NO NO NO